In [0]:
import tensorflow as tf

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.size']=14

---
---

# <center> CIFAR 10

In [2]:
# Import Cifar data
Cifar = tf.keras.datasets.cifar10
# Creating training and testing datasets
(x_train, y_train), (x_test, y_test) = Cifar.load_data()

170500096/170498071 [==============================] - 4s 0us/step


In [3]:
x_train = x_train/255
x_test = x_test/255
x_train.shape

(50000, 32, 32, 3)

In [4]:
y_train.shape

(50000, 1)

In [0]:
from tensorflow.keras.utils import to_categorical

In [0]:
y_test = to_categorical(y_test)
y_train = to_categorical(y_train)

## Without Transfer Learning

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Activation
from tensorflow.keras.models import Model

In [0]:
model= Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5), activation='relu', input_shape = (32, 32, 3)))
model.add(MaxPooling2D(pool_size = (2,2), strides=2))
model.add(Conv2D(filters=64, kernel_size = 5, activation='relu'))
model.add(MaxPooling2D(pool_size = 2, strides=2))
model.add(Conv2D(filters=120, kernel_size = 5, activation='relu'))
model.add(Flatten())
model.add(Dense(84, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
%%time
history = model.fit(x_train, y_train, batch_size=1024, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
49/49 [==============================] - 2s 31ms/step - loss: 1.9399 - accuracy: 0.2890 - val_loss: 1.7127 - val_accuracy: 0.3743
Epoch 2/5
49/49 [==============================] - 1s 24ms/step - loss: 1.6149 - accuracy: 0.4163 - val_loss: 1.5055 - val_accuracy: 0.4603
Epoch 3/5
49/49 [==============================] - 1s 25ms/step - loss: 1.4643 - accuracy: 0.4745 - val_loss: 1.4053 - val_accuracy: 0.4984
Epoch 4/5
49/49 [==============================] - 1s 23ms/step - loss: 1.3817 - accuracy: 0.5088 - val_loss: 1.3430 - val_accuracy: 0.5224
Epoch 5/5
49/49 [==============================] - 1s 24ms/step - loss: 1.2903 - accuracy: 0.5420 - val_loss: 1.2608 - val_accuracy: 0.5586
CPU times: user 6.99 s, sys: 1.83 s, total: 8.81 s
Wall time: 13.4 s


---
---
---

## Transfer Learning with VGG-16:
* #### Training entire architecture with trained weights as starting point (Not recommended for small datasets)

#### Note CIFAR uses smaller(32, 32, 3) images compared to original model (224, 224, 3)

In [0]:
from tensorflow.keras.applications.vgg16 import VGG16

In [0]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape = (32, 32, 3))

In [0]:
# base_model.summary()

In [0]:
model = Sequential([base_model])
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [0]:
# model.summary()

In [15]:
%%time
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=1024, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
49/49 [==============================] - 12s 247ms/step - loss: 2.1997 - accuracy: 0.1524 - val_loss: 1.9354 - val_accuracy: 0.2149
Epoch 2/5
49/49 [==============================] - 10s 195ms/step - loss: 1.8584 - accuracy: 0.2441 - val_loss: 1.7949 - val_accuracy: 0.2737
Epoch 3/5
49/49 [==============================] - 10s 196ms/step - loss: 1.6953 - accuracy: 0.3182 - val_loss: 1.5762 - val_accuracy: 0.3909
Epoch 4/5
49/49 [==============================] - 10s 195ms/step - loss: 1.4612 - accuracy: 0.4205 - val_loss: 1.3332 - val_accuracy: 0.4947
Epoch 5/5
49/49 [==============================] - 10s 195ms/step - loss: 1.1728 - accuracy: 0.5487 - val_loss: 1.0494 - val_accuracy: 0.6060
CPU times: user 27 s, sys: 13.8 s, total: 40.9 s
Wall time: 54.6 s


---
---
---

## Transfer Learning with VGG-16:
* #### Training only the top layers (Feature Extraction)

In [0]:
from tensorflow.keras.applications.vgg16 import VGG16

In [0]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape = (32, 32, 3))

In [0]:
# base_model.summary()

In [0]:
base_model.trainable = False

In [0]:
model = Sequential([base_model])
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [41]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
flatten_5 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                32832     
_________________________________________________________________
dense_11 (Dense)             (None, 10)                650       
Total params: 14,748,170
Trainable params: 33,482
Non-trainable params: 14,714,688
_________________________________________________________________


In [22]:
%%time
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=1024, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
49/49 [==============================] - 4s 81ms/step - loss: 1.9058 - accuracy: 0.3356 - val_loss: 1.6067 - val_accuracy: 0.4489
Epoch 2/5
49/49 [==============================] - 4s 78ms/step - loss: 1.4827 - accuracy: 0.4911 - val_loss: 1.4195 - val_accuracy: 0.5112
Epoch 3/5
49/49 [==============================] - 4s 79ms/step - loss: 1.3577 - accuracy: 0.5328 - val_loss: 1.3488 - val_accuracy: 0.5322
Epoch 4/5
49/49 [==============================] - 4s 78ms/step - loss: 1.2965 - accuracy: 0.5543 - val_loss: 1.3027 - val_accuracy: 0.5476
Epoch 5/5
49/49 [==============================] - 4s 79ms/step - loss: 1.2574 - accuracy: 0.5664 - val_loss: 1.2777 - val_accuracy: 0.5582
CPU times: user 5.67 s, sys: 1.1 s, total: 6.77 s
Wall time: 20.6 s


---
---
---

## Transfer Learning with VGG-16: 
* #### Training more layers (Fine Tuning)

In [0]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape = (32, 32, 3))

In [0]:
VGG16?

In [0]:
# base_model.summary()

In [43]:
for i, layer in enumerate(base_model.layers):
    print(i, layer.name)

0 input_6
1 block1_conv1
2 block1_conv2
3 block1_pool
4 block2_conv1
5 block2_conv2
6 block2_pool
7 block3_conv1
8 block3_conv2
9 block3_conv3
10 block3_pool
11 block4_conv1
12 block4_conv2
13 block4_conv3
14 block4_pool
15 block5_conv1
16 block5_conv2
17 block5_conv3
18 block5_pool


In [0]:
for layer in base_model.layers[:16]:
       layer.trainable = False

In [45]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [0]:
model = Sequential([base_model])
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [47]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten_6 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 64)                32832     
_________________________________________________________________
dense_13 (Dense)             (None, 10)                650       
Total params: 14,748,170
Trainable params: 4,753,098
Non-trainable params: 9,995,072
_________________________________________________________________


In [30]:
%%time
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=1024, epochs=5, validation_data=(x_test, y_test))

Epoch 1/5
49/49 [==============================] - 5s 95ms/step - loss: 1.4937 - accuracy: 0.4788 - val_loss: 1.1275 - val_accuracy: 0.6074
Epoch 2/5
49/49 [==============================] - 4s 92ms/step - loss: 1.0042 - accuracy: 0.6502 - val_loss: 1.0142 - val_accuracy: 0.6478
Epoch 3/5
49/49 [==============================] - 5s 92ms/step - loss: 0.8839 - accuracy: 0.6900 - val_loss: 0.9196 - val_accuracy: 0.6768
Epoch 4/5
49/49 [==============================] - 4s 92ms/step - loss: 0.7985 - accuracy: 0.7194 - val_loss: 0.8853 - val_accuracy: 0.6882
Epoch 5/5
49/49 [==============================] - 5s 93ms/step - loss: 0.7399 - accuracy: 0.7380 - val_loss: 0.8762 - val_accuracy: 0.6989
CPU times: user 7.9 s, sys: 2.36 s, total: 10.3 s
Wall time: 24 s


---
---
---

## End